In [3]:
# Parallel processing with ThreadPoolExecutor
def process_row(row, processor):
    return pd.Series(processor.get_metrics(row))


def process_df_stopsales(df_stopsales, processor):
    with ThreadPoolExecutor() as executor:
        results = list(
            executor.map(
                lambda row: process_row(row, processor), df_stopsales.to_dict("records")
            )
        )
    return pd.DataFrame(results)


# Usage
metrics_processor = MetricsProcessor()
df_stopsales[
    [
        "revenue_last_12_months",
        "average_per_week",
        "revenue_post_rule",
        "monthly_revenue_last_12_months",
        "weekday_revenue_last_12_months",
    ]
] = process_df_stopsales(df_stopsales, metrics_processor)

end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.4f} seconds")
display(df_stopsales.head())

levels_rules__ps__t__gt
levels_rules__mel__t__gt
levels_rules__cid__t__gt
levels_rules__bod__t__gt
levels_rules__dow__t__gt
levels_rules__room__t__gt
levels_rules__non__t__gt
levels_rules__hou__t__gt
levels_rules__mrk__t__gt
levels_rules__cp__gt
levels_rules__rat__gt
levels_rules__rel__gt
levels_rules__age__gt
